<a href="https://colab.research.google.com/github/baharvand79/Parkinson-s-Voice-Analyzer/blob/main/pretrain_cnn_inception.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==============================================================================
#  AUTOMATIC DEPENDENCY INSTALLER
# ==============================================================================
import subprocess
import sys
import importlib

# A dictionary of packages to check, mapping the import name to the pip package name
packages = {
    'pandas': 'pandas',
    'numpy': 'numpy',
    'tensorflow': 'tensorflow',
    'sklearn': 'scikit-learn',
    'matplotlib': 'matplotlib'
}

print("Checking for required packages...")

for import_name, pip_name in packages.items():
    try:
        # Try to import the package
        importlib.import_module(import_name)
        print(f"{import_name} is already installed.")
    except ImportError:
        # If it fails, install it using pip
        print(f"{import_name} not found. Installing...")
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", pip_name])
            print(f"Successfully installed {pip_name}.")
        except subprocess.CalledProcessError as e:
            print(f"Failed to install {pip_name}. Error: {e}")
            # Exit if a critical package fails to install
            sys.exit(1)

print("\nAll dependencies are satisfied. Starting the main script...\n")

Checking for required packages...
pandas is already installed.
numpy is already installed.
tensorflow is already installed.
sklearn is already installed.
matplotlib is already installed.

All dependencies are satisfied. Starting the main script...



In [ ]:
# ==============================================================================
#  IMPORTING DEPENDENCIES
# ==============================================================================
import pandas as pd
import numpy as np
import tensorflow as tf
import timeit
import datetime
import math
import os # Often useful for path handling
from matplotlib import pyplot as plt

# Keras imports from the tensorflow.keras path
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Activation, Dense, Dropout, Flatten, BatchNormalization, Conv2D, MaxPooling2D, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications.inception_v3 import InceptionV3

# Scikit-learn imports
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score

In [ ]:
from google.colab import drive

# === 1. ENVIRONMENT SETUP ===
# Mount Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# === 2. CONFIGURATION ===
# --- Main Paths ---
GDRIVE_BASE_PATH = '/content/drive/MyDrive/Scripts/pretrained_cnn' # CHANGE to your project's base folder on Drive
DATASET_NAME = "UAMS"
SPECTROGRAM_TYPE = "linear" # Change to "mel" to use mel-spectrograms

# --- Auto-generated Paths ---
# Path to the input spectrogram images
INPUT_DATA_PATH = os.path.join(GDRIVE_BASE_PATH, 'spectrogram_data', f'{DATASET_NAME}_{SPECTROGRAM_TYPE}')
# Path where all outputs (models, logs, plots) will be saved
OUTPUT_DIR = os.path.join(GDRIVE_BASE_PATH, 'training_output', f'{DATASET_NAME}_{SPECTROGRAM_TYPE}_output')

# --- Hyperparameters ---
IMG_ROWS = 600
IMG_COLS = 600
BATCH_SIZE = 4
EPOCHS = 10
N_RUNS = 10 # Set to 100 for full experiment

# --- Create output directory if it doesn't exist ---
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"Input data will be read from: {INPUT_DATA_PATH}")
print(f"All outputs will be saved to: {OUTPUT_DIR}")




Input data will be read from: /content/drive/MyDrive/Scripts/pretrained_cnn/spectrogram_data/UAMS_linear
All outputs will be saved to: /content/drive/MyDrive/Scripts/pretrained_cnn/training_output/UAMS_linear_output


In [ ]:

# === 3. DATA GENERATORS ===
# Note: Using validation_split requires specifying subset='training' or 'validation'
data_generator = ImageDataGenerator(
    rescale=1. / 255,
    fill_mode='nearest',
    validation_split=0.3  # Splitting data into 70% training, 30% validation
)

train_generator = data_generator.flow_from_directory(
    INPUT_DATA_PATH,
    target_size=(IMG_ROWS, IMG_COLS),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
    subset='training' # Set as training data
)

validation_generator = data_generator.flow_from_directory(
    INPUT_DATA_PATH,
    target_size=(IMG_ROWS, IMG_COLS),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False, # Important: Do not shuffle validation data
    subset='validation' # Set as validation data
)


Found 57 images belonging to 2 classes.
Found 24 images belonging to 2 classes.


In [ ]:
# === 4. TRAINING LOOP ===
start_time = timeit.default_timer()
all_aucs = []
best_run_auc = -1
best_run_info = {}

for run in range(N_RUNS):
    print("-" * 50)
    print(f"Starting Run: {run + 1}/{N_RUNS}")
    print("-" * 50)

    # --- Build the Model (re-initialized each run) ---
    base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(IMG_ROWS, IMG_COLS, 3), pooling='avg')
    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = BatchNormalization()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.2)(x)
    predictions = Dense(2, activation='softmax')(x) # Use softmax for categorical_crossentropy

    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(
        loss='categorical_crossentropy',
        optimizer=Adam(learning_rate=0.001),
        metrics=['accuracy']
    )

    # --- Callbacks ---
    checkpoint_path = os.path.join(OUTPUT_DIR, f"{DATASET_NAME}_{SPECTROGRAM_TYPE}_run-{run+1:02d}_best_model.h5")
    model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_path,
        save_best_only=True,
        save_weights_only=False,
        monitor='val_accuracy', # Monitor validation accuracy
        mode='max'
    )

    # --- Train the Model ---
    history = model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // BATCH_SIZE,
        epochs=EPOCHS,
        validation_data=validation_generator,
        validation_steps=validation_generator.samples // BATCH_SIZE,
        callbacks=[model_checkpoint],
        verbose=2
    )

    # --- Evaluation for the current run ---
    # Load the best model from this run for evaluation
    best_model_this_run = tf.keras.models.load_model(checkpoint_path)
    Y_pred = best_model_this_run.predict(validation_generator)
    y_pred_classes = np.argmax(Y_pred, axis=1)

    fpr, tpr, _ = roc_curve(validation_generator.classes, y_pred_classes)
    run_auc = auc(fpr, tpr)
    all_aucs.append(run_auc)
    print(f"\nRun {run + 1} AUC: {run_auc:.4f}")

    # --- Track the best run overall ---
    if run_auc > best_run_auc:
        best_run_auc = run_auc
        best_run_info = {
            'run_number': run + 1,
            'model_path': checkpoint_path,
            'y_pred': Y_pred,
            'fpr': fpr,
            'tpr': tpr
        }

--------------------------------------------------
Starting Run: 1/10
--------------------------------------------------
Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


14/14 - 65s - 5s/step - accuracy: 0.3962 - loss: 2.5413 - val_accuracy: 0.5833 - val_loss: 0.9682
Epoch 2/10


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


14/14 - 23s - 2s/step - accuracy: 0.2500 - loss: 2.4306 - val_accuracy: 0.5833 - val_loss: 1.0092
Epoch 3/10
14/14 - 55s - 4s/step - accuracy: 0.7547 - loss: 1.6398 - val_accuracy: 0.3750 - val_loss: 0.8465
Epoch 4/10
14/14 - 18s - 1s/step - accuracy: 0.5000 - loss: 1.4413 - val_accuracy: 0.3750 - val_loss: 0.8961
Epoch 5/10
14/14 - 49s - 4s/step - accuracy: 0.7547 - loss: 1.1482 - val_accuracy: 0.5833 - val_loss: 0.9108
Epoch 6/10
14/14 - 18s - 1s/step - accuracy: 0.7500 - loss: 0.2875 - val_accuracy: 0.5833 - val_loss: 0.9591
Epoch 7/10


...

14/14 - 59s - 4s/step - accuracy: 0.4528 - loss: 2.1696 - val_accuracy: 0.5833 - val_loss: 0.6513
Epoch 2/10


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


14/14 - 21s - 1s/step - accuracy: 0.5000 - loss: 2.2345 - val_accuracy: 0.6250 - val_loss: 0.6752
Epoch 3/10


14/14 - 61s - 4s/step - accuracy: 0.7170 - loss: 2.4953 - val_accuracy: 0.6667 - val_loss: 0.6171
Epoch 4/10
14/14 - 23s - 2s/step - accuracy: 0.5000 - loss: 2.5452 - val_accuracy: 0.6667 - val_loss: 0.6710
Epoch 5/10
14/14 - 56s - 4s/step - accuracy: 0.8302 - loss: 1.6351 - val_accuracy: 0.4167 - val_loss: 1.1563
Epoch 6/10
14/14 - 18s - 1s/step - accuracy: 0.7500 - loss: 0.4068 - val_accuracy: 0.4583 - val_loss: 1.0417
Epoch 7/10
14/14 - 50s - 4s/step - accuracy: 0.7358 - loss: 0.9602 - val_accuracy: 0.5833 - val_loss: 0.5715
Epoch 8/10
14/14 - 23s - 2s/step - accuracy: 0.5000 - loss: 1.0056 - val_accuracy: 0.5833 - val_loss: 0.5999
Epoch 9/10
14/14 - 59s - 4s/step - accuracy: 0.9057 - loss: 0.2929 - val_accuracy: 0.4167 - val_loss: 1.0727
Epoch 10/10
14/14 - 18s - 1s/step - accuracy: 1.0000 - loss: 2.3089e-04 - val_accuracy: 0.4167 - val_loss: 1.0126


6/6 ━━━━━━━━━━━━━━━━━━━━ 18s 3s/step

Run 10 AUC: 0.6667


In [ ]:

# === 5. FINAL REPORTING ===
print("\n" + "=" * 50)
print("Training Complete. Generating Final Report.")
print("=" * 50)

# --- Overall Performance ---
avg_auc = np.mean(all_aucs)
std_auc = np.std(all_aucs)
print(f"Average AUC over {N_RUNS} runs: {avg_auc:.4f} (std: {std_auc:.4f})")
print(f"Best run was Run #{best_run_info['run_number']} with AUC: {best_run_auc:.4f}")

# --- Load the best model from the best run ---
print(f"Loading best overall model from: {best_run_info['model_path']}")
best_overall_model = tf.keras.models.load_model(best_run_info['model_path'])

# --- Confusion Matrix and Classification Report ---
y_pred_best = np.argmax(best_run_info['y_pred'], axis=1)
report_str = "--- Confusion Matrix ---\n"
report_str += str(confusion_matrix(validation_generator.classes, y_pred_best))
report_str += "\n\n--- Classification Report ---\n"
target_names = list(train_generator.class_indices.keys())
report_str += classification_report(validation_generator.classes, y_pred_best, target_names=target_names)

print(report_str)

# --- Save the Final Report to a File ---
report_path = os.path.join(OUTPUT_DIR, "final_classification_report.txt")
with open(report_path, "w") as f:
    f.write(f"Final Report for {DATASET_NAME} - {SPECTROGRAM_TYPE}\n\n")
    f.write(f"Average AUC over {N_RUNS} runs: {avg_auc:.4f} (std: {std_auc:.4f})\n")
    f.write(f"Best run was Run #{best_run_info['run_number']} with AUC: {best_run_auc:.4f}\n\n")
    f.write(report_str)
print(f"Final report saved to: {report_path}")

# --- Plot and Save ROC Curve for the Best Run ---
plt.figure(figsize=(8, 6))
plt.plot(best_run_info['fpr'], best_run_info['tpr'], color='darkorange', lw=2,
         label=f'ROC curve (area = {best_run_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title(f'Receiver Operating Characteristic (Best Run) - {DATASET_NAME} {SPECTROGRAM_TYPE}')
plt.legend(loc="lower right")
roc_plot_path = os.path.join(OUTPUT_DIR, "best_run_ROC_curve.png")
plt.savefig(roc_plot_path)
plt.show()
print(f"ROC curve plot saved to: {roc_plot_path}")

# --- Record Runtime ---
stop_time = timeit.default_timer()
print(f"\nTotal RunTime: {round(stop_time - start_time, 2)} Seconds")